In [ ]:
import nltk
import timeit
!pip install datasets

from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 13.1 MB/s eta 0:00:00


In [ ]:
# select a year between 1810 and 1963 and set variable

my_year = "1953"

# set number of articles (keep this small - it's slow)
num_articles = 8

#  Download data for selection
dataset = load_dataset("dell-research-harvard/AmericanStories", "subset_years", year_list=[my_year])

# Get the first n articles from that year
# instantiate the counter
i=0
# instantiate the string
my_articles = ''
# loop through each article for that year
for article in dataset[my_year]:
    my_articles += article.get('article')
    i+=1
    #if the counter is greater than num_articles-1, stop looping
    if i>(num_articles-1): break

#validate results by checking first 1000 characters
print(my_articles[:1001])

Only taking a subset of years. Change name to 'all_years' to use all years in the dataset.
{'1953': 'https://huggingface.co/datasets/dell-research-harvard/AmericanStories/resolve/main/faro_1953.tar.gz'}


Generating 1953 split: 0 examples [00:00, ? examples/s]

Loading associated
never cared much about how
she looked. Now she realizes
that was an error.


Vanessa's husband, a very
successful plastic surgeon, finds
himself in the anomalous posi-
tion of having to commute
more often than his wife ever
did-between the two coasts.


Its his own fault. He
picked "The Itch" as least
likely to succeed from a stack
of scripts we had to choose
from last summer" smiles VAN
essa. "You see, he wanted me
on Broadway, but not too long.


But my husbands prac-
tice requires his presence in
New York as we'll as in L. A.
anyway" Vanessa explains,
and continues in a lighter
vein, 80 his trips East are
not completely wasted"


Vanessa and the doctor have
home in Holmby Hills. "All
windows from top to bottom.
no plaster. just rock and wood
rough finish on all the va
rious exotic woods.


In New York, VanesSa lives
high in the sky, facing Gen.
trail Park. But when "The
Seven Year Itch" has run it's
course, she plans to return
to Hollywood.


Between shows, VaneSs

In [ ]:
#remove new line and other formatting characters
for char in ["\n", "\r", "\d", "\t"]:
    my_articles = my_articles.replace(char, " ")
my_articles[:1001]

'never cared much about how she looked. Now she realizes that was an error.   Vanessa\'s husband, a very successful plastic surgeon, finds himself in the anomalous posi- tion of having to commute more often than his wife ever did-between the two coasts.   Its his own fault. He picked "The Itch" as least likely to succeed from a stack of scripts we had to choose from last summer" smiles VAN essa. "You see, he wanted me on Broadway, but not too long.   But my husbands prac- tice requires his presence in New York as we\'ll as in L. A. anyway" Vanessa explains, and continues in a lighter vein, 80 his trips East are not completely wasted"   Vanessa and the doctor have home in Holmby Hills. "All windows from top to bottom. no plaster. just rock and wood rough finish on all the va rious exotic woods.   In New York, VanesSa lives high in the sky, facing Gen. trail Park. But when "The Seven Year Itch" has run it\'s course, she plans to return to Hollywood.   Between shows, VaneSsa ap- pears on 

In [ ]:
# add lemma package from nltk

from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
wn_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# put in l

%%time

#split the string on spaces nand return a list
whitespace_tokens = my_articles.split(' ')

#confirm results
whitespace_tokens[:16]

CPU times: user 251 µs, sys: 12 µs, total: 263 µs
Wall time: 269 µs


['never',
 'cared',
 'much',
 'about',
 'how',
 'she',
 'looked.',
 'Now',
 'she',
 'realizes',
 'that',
 'was',
 'an',
 'error.',
 '',
 '']

In [ ]:
# this does not appear to be working correctly, the only adjustment it made was to remove the "s" at the end of "was"

my_lemmas = []
for word in whitespace_tokens:
    w = wn_lemmatizer.lemmatize(word)
    my_lemmas.append(w)
my_lemmas[:16]

['never',
 'cared',
 'much',
 'about',
 'how',
 'she',
 'looked.',
 'Now',
 'she',
 'realizes',
 'that',
 'wa',
 'an',
 'error.',
 '',
 '']

In [19]:
!pip install bpe
from bpe import Encoder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.7/420.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 38.7 MB/s eta 0:00:00


In [21]:
%%time

# calling the Encoder algorithm
# we've specified 100 token vocab and 95% to be tokenized
# the other 5% is transformed into UNK
encoder = Encoder(100, pct_bpe=0.95)
encoder.fit(whitespace_tokens)

#print(encoder.tokenize(my_articles))

print(next(encoder.inverse_transform(encoder.transform([my_articles]))))

never cared much about how she looked . now she reali__unkes that was an error . vanessa ' s husband , a very successful plastic surgeon , finds himself in the anomalous posi - tion of having to commute more often than his wife ever did - between the two coasts . its his own fault . he picked " the itch " as least likely to succeed from a stack of scripts we had to choose from last summer " smiles van essa . " you see , he wanted me on broadway , but not too long . but my husbands prac - tice re__unkuires his presence in new york as we ' ll as in l . a . anyway " vanessa e__unkplains , and continues in a lighter vein , __unk__unk his trips east are not completely wasted " vanessa and the doctor have home in holmby hills . " all windows from top to bottom . no plaster . __unkust rock and wood rough finish on all the va rious e__unkotic woods . in new york , vanessa lives high in the sky , facing gen . trail park . but when " the seven year itch " has run it ' s course , she plans to ret